In [ ]:
# Ensure you deployed mlflow
%env MLFLOW_TRACKING_URI=http://localhost:5000
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ACCESS_KEY_ID=minio
%env AWS_SECRET_ACCESS_KEY=minio123

# System libraries
import sys
import time
import os.path
from pathlib import Path

# Import DS librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import mlflow

# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.python.saved_model import signature_constants

import sys
sys.path.append("/home/thomas/workspace/ml_project_example/")
from src.domain.model import build_model

In [ ]:
def generate_data_for_training(data_path, batch_size):

    image_dir = Path(data_path)
    # Get filepaths and labels
    filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.PNG'))
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    # Concatenate filepaths and labels
    image_df = pd.concat([filepaths, labels], axis=1)

    # Separate in train and test data
    train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

    # Train data generator
    train_generator = ImageDataGenerator(validation_split=0.2)
    # Test data generator
    test_generator = ImageDataGenerator()

    # Split the data into three categories.
    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=42,
        subset='training'
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=42,
        subset='validation'
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=False
    )

    return train_images, val_images, test_images, test_df

In [ ]:
# Set path to data
data_path = "../data"

# You should ensure you have a local mlflow server running
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("sea_animals_classification")

with mlflow.start_run(run_name="Exploration"):

    # hyperparams
    denses = [256,256,19]
    dropout = [0.2, 0.2]
    adam_param = 0.00001
    nb_epochs = 20
    batch_size = 8
    input_shape = (224, 224, 3)

    # Logging hyper parameters to mlflow
    mlflow.log_param("adam", adam_param)
    mlflow.log_param("dropout", dropout)
    mlflow.log_param("nb_dense", denses)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("nb_epochs", nb_epochs)
    mlflow.log_param("input_shape", input_shape)
    print("Training parameters logged to tracking server.")
    
    # Data Generators
    train_images, val_images, test_images, test_df = generate_data_for_training(data_path, batch_size)
    
    # Model definition
    model = build_model(input_shape, denses)

    model.compile(
        optimizer=Adam(adam_param),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    start_training_time = time.time()
    history = model.fit(
        train_images,
        steps_per_epoch=len(train_images),
        validation_data=val_images,
        validation_steps=len(val_images),
        epochs=nb_epochs
    )
    elapsed_training_time = time.time() - start_training_time

    # Evaluate model
    results = model.evaluate(test_images, verbose=1)

    mlflow.log_metric("Training time", elapsed_training_time)
    mlflow.log_metric("Training accuracy", history.history['accuracy'][0]*100)
    mlflow.log_metric("Validation accuracy", history.history['accuracy'][1]*100)
    mlflow.log_metric("Test accuracy", results[1]*100)
    print("Metrics logged to tracking server.")

    tf.keras.models.save_model(model, "./model")
    print("Model saved locally")

    # Log training script for audit purposes
    try:
        mlflow.log_artifacts("./training.ipynb")
        mlflow.log_artifacts("../src/domain/model.py")
        print("Training & model scripts logged as artifacts.")
    except Exception as e:
        print("Impossible to log artifacts: {}".format(e))

    # Log model for reuse & comparison
    try:
        tag=[tf.compat.v1.saved_model.tag_constants.SERVING]
        key=signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
        mlflow.tensorflow.log_model(tf_saved_model_dir="./model",
                                tf_meta_graph_tags=tag,
                                tf_signature_def_key=key,
                                artifact_path="model",
                                registered_model_name="MobileNetV2")
        print("Model sent to registry.")
    except Exception as e:
        print("Impossible to log model to registry: {}".format(e))

In [ ]:
from helper_functions import make_confusion_matrix, plot_loss_curves

# Plot loss curves
plot_loss_curves(history)

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
# Output confusion matrix
make_confusion_matrix(y_test, pred, list(labels.values()))

In [ ]:
# Display 25 random pictures from the dataset with their labels
random_index = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_index[i]]))
    if test_df.Label.iloc[random_index[i]] == pred[random_index[i]]:
      color = "green"
    else:
      color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}", color=color)
plt.show()
plt.tight_layout()